<a href="https://colab.research.google.com/github/AdaobiNwafor/Coursera_Capstone_/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook will be used for the capstone project in the IBM Data Science Professional Certificate cCoursera.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('hello capstone project course!')

hello capstone project course!


# Importing and cleaning the Postal codes of canada 

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
print('modules imported')

modules imported


In [13]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

contents = []
table = soup.find("table")
for row in table.findAll('td'):
  cell = {}
  if row.span.text == 'Not assigned':
    pass
  else:
      cell['PostalCode'] = row.p.text[:3]
      cell['Borough'] = (row.span.text).split('(')[0]
      cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
      contents.append(cell)


# table_rows = table.tbody.find_all("tr")
# for tr in table_rows:
#     td = tr.find_all("td")
#     row = [tr.text for tr in td]
    
#     # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#     if row != [] and row[1] != "Not assigned":
#         # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
#         if "Not assigned" in row[2]: 
#             row[2] = row[1]
#         res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
df.shape

(103, 3)

# Creating the Longitude and Latitude dataframe

In [50]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [58]:
coord1 = pd.read_csv('Geospatial_Coordinates.csv')   

In [62]:
coord = coord1[['Latitude', 'Longitude']]
coord

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476
...,...,...
98,43.706876,-79.518188
99,43.696319,-79.532242
100,43.688905,-79.554724
101,43.739416,-79.588437


In [66]:
canada = pd.concat([df, coord], axis=1)
canada

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East Toronto Business,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437


# Creating a map of Canada neighbourhood

In [73]:
import folium
import geopy
from  geopy.geocoders import Nominatim
print('modules imported')

modules imported


In [79]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
import matplotlib.cm as cm
import matplotlib.colors as colors
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [83]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canada['Latitude'], canada['Longitude'], canada['Borough'], canada['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [85]:
north_york = canada[canada['Borough'] == 'North York'].reset_index(drop=True)
north_york.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
3,M3B,North York,Don Mills North,43.711112,-79.284577
4,M6B,North York,Glencairn,43.757410,-79.273304


In [88]:
address = 'North York, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [89]:
map_nyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york['Latitude'], north_york['Longitude'], north_york['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyork)  
    
map_nyork



## Setting parameters



In [99]:
CLIENT_ID = 'LVINK2Q5O52GAXXQ3SN3RSCQSIXL4KUIV0CSVFTC4RNBWF1C' # your Foursquare ID
CLIENT_SECRET = 'NRSECBZA2BFXIZGOWP0REIAF3VD4HQZ0MVAGCIES1ZQJVMVE' # your Foursquare Secret
VERSION = '20210621' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LVINK2Q5O52GAXXQ3SN3RSCQSIXL4KUIV0CSVFTC4RNBWF1C
CLIENT_SECRET:NRSECBZA2BFXIZGOWP0REIAF3VD4HQZ0MVAGCIES1ZQJVMVE


In [100]:
# Getting the neighbourhood in North York
north_york.loc[0, 'Neighborhood']

'Parkwoods'

In [101]:
# Getting parkwood lng, lat
parkwood_lat = north_york.loc[0, 'Latitude'] # neighborhood latitude value
parkwood_lng = north_york.loc[0, 'Longitude'] # neighborhood longitude value

parkwoods = north_york.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(parkwoods, 
                                                               parkwood_lat, 
                                                               parkwood_lng))

Latitude and longitude values of Parkwoods are 43.806686299999996, -79.19435340000001.


# Getting the url

```
# This is formatted as code
```



In [110]:
radius = 1000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    parkwood_lat, 
    parkwood_lng, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=LVINK2Q5O52GAXXQ3SN3RSCQSIXL4KUIV0CSVFTC4RNBWF1C&client_secret=NRSECBZA2BFXIZGOWP0REIAF3VD4HQZ0MVAGCIES1ZQJVMVE&v=20210621&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

### Finding the venues in a 1000 mile radius

In [111]:
import requests 
import json
from pandas.io.json import json_normalize
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d0c6fd8175de04db2002cd'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c6ac7de35d3be9a50bf2206-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10a951735',
         'name': 'Bank',
         'pluralName': 'Banks',
         'primary': True,
         'shortName': 'Bank'}],
       'id': '4c6ac7de35d3be9a50bf2206',
       'location': {'address': '865 MILNER AVE',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'Morningside',
        'distance': 906,
        'formattedAddress': ['865 MILNER AVE (Morningside)',
         'Scarborough ON M1B 5N6',
         'Canada'],
        'labeledLatLngs': [{'label': 'di

In [112]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [113]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.798782,-79.197090
1,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Harvey's,Restaurant,43.800020,-79.198307
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
